# 初识tensorflow
此简短介绍使用 Keras 进行以下操作：
1. 加载一个预构建的数据集。
2. 构建对图像进行分类的神经网络机器学习模型。
3. 训练此神经网络。
4. 评估模型的准确率。

### 1. 设置 TensorFlow
首先将 TensorFlow 导入到您的程序：

In [30]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.11.0


### 2. 加载数据集
加载并准备MNIST数据集。图像的像素值范围从0到255。通过将这些值除以255.0，将这些值缩放到0到1的范围。这也将示例数据从整数转换为浮点数：

In [31]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

### 3.构建机器学习模型
通过堆叠层来构建 tf.keras.Sequential 模型。

In [60]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
    ])

对于每个样本，模型都会返回一个包含 logits 或 log-odds 分数的向量，每个类一个。

In [61]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.43538293,  0.8154418 ,  0.71953803,  0.27638298,  0.10783938,
         0.60322917, -0.48105052, -0.13477172, -0.8981102 , -0.5059797 ]],
      dtype=float32)

tf.nn.softmax 函数将这些 logits 转换为每个类的概率：

In [62]:
tf.nn.softmax(predictions).numpy()

array([[0.0551911 , 0.19279478, 0.17516398, 0.11245662, 0.09501397,
        0.15593101, 0.05272733, 0.07454573, 0.03474634, 0.05142914]],
      dtype=float32)

注：可以将 tf.nn.softmax 嵌入到网络最后一层的激活函数中。虽然这可以使模型输出更易解释，但不建议使用这种方式，因为在使用 softmax输出时，不可能为所有模型提供精确且数值稳定的损失计算。

使用 losses.SparseCategoricalCrossentropy 为训练定义损失函数，它会接受 logits 向量和 True 索引，并为每个样本返回一个标量损失。

In [63]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

此损失等于 true 类的负对数概率：如果模型确定类正确，则损失为零。

这个未经训练的模型给出的概率接近随机（每个类为 1/10），因此初始损失应该接近 -tf.math.log(1/10) ~= 2.3。

In [65]:
loss_fn(y_train[:1], predictions).numpy()

1.8583417

在开始训练之前，使用 Keras Model.compile 配置和编译模型。将 optimizer 类设置为 adam，将 loss 设置为您之前定义的 loss_fn 函数，并通过将 metrics 参数设置为 accuracy 来指定要为模型评估的指标。

In [37]:
model.compile(
    optimizer='adam',
    loss=loss_fn,
    metrics=['accuracy']
)

### 4. 训练并评估模型
使用 Model.fit 方法调整您的模型参数并最小化损失：

In [38]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.2970 - accuracy: 0.9136
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1427 - accuracy: 0.9573
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1060 - accuracy: 0.9685
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0859 - accuracy: 0.9732
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0759 - accuracy: 0.9769


Model.evaluate 方法通常在 "Validation-set" 或 "Test-set" 上检查模型性能。

In [39]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 4s - loss: 0.0731 - accuracy: 0.9773 - 4s/epoch - 14ms/step


[0.07311903685331345, 0.9772999882698059]

现在，这个照片分类器的准确度已经达到 98%。想要了解更多，请阅读 TensorFlow 教程。

如果您想让模型返回概率，可以封装经过训练的模型，并将 softmax 附加到该模型：

In [40]:
probability_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Softmax()
])

In [41]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[5.13888168e-08, 8.23802626e-09, 7.21608149e-06, 2.02014911e-04,
        1.28073796e-09, 3.12419570e-06, 1.61449359e-12, 9.99783218e-01,
        2.35225457e-07, 4.19382923e-06],
       [8.61052341e-10, 6.29989809e-05, 9.99917865e-01, 1.89470120e-05,
        1.07603402e-15, 5.67801877e-08, 3.71941811e-09, 1.97910027e-14,
        6.06561983e-08, 5.58183838e-12],
       [1.07529399e-06, 9.99490499e-01, 3.41384221e-05, 7.21691413e-06,
        6.11745272e-05, 2.96484541e-06, 1.07345941e-05, 2.78506835e-04,
        1.13107395e-04, 5.04392233e-07],
       [9.99964952e-01, 4.65795535e-09, 2.02851043e-05, 9.53296194e-07,
        8.93702889e-09, 6.58222120e-07, 1.16968904e-05, 7.68486416e-07,
        1.13966721e-08, 7.50930099e-07],
       [5.03222464e-06, 4.06263929e-08, 1.06649964e-06, 4.14982182e-08,
        9.97832239e-01, 1.97049033e-07, 1.81184896e-05, 4.13856433e-05,
        3.98354956e-07, 2.10159924e-03]], dtype=float32)>